## Lab. 12

### Solve the following problem using Genetic Algorithms:


Problem: Weighted N-Queen Problem


You are given an N×N chessboard, and each cell of the board has an associated weight. Your task is to find a valid placement of N queens such that the total weight of the queens is maximized, and no two queens threaten each other.





In the traditional N-Queen Problem, the goal is to place N queens on an N×N chessboard in such a way that no two queens threaten each other. In this variation, we introduce weights to the queens and aim to find a placement that maximizes the total weight of the queens while satisfying the constraint of non-threatening positions.


Constraints:

1. There should be exactly one queen in each row and each column.
2. No two queens should be placed in the same diagonal, i.e., they should not threaten each other.
3. The placement should maximize the total weight of the queens.


Representation:

Use a permutation-based representation. Each permutation represents the column position of the queen for each row. 

For example, if N=4, a valid permutation [2, 4, 1, 3] indicates that the queen in the first row is placed in column 2, the queen in the second row is placed in column 4, and so on.


Genetic Algorithm Steps:

1. *Initialization*: Generate an initial population of permutations randomly.

2. *Fitness Evaluation*: Evaluate the fitness of each permutation by calculating the total weight of the queens while considering the non-threatening positions.

3. *Selection*: Select a subset of permutations from the population based on their fitness, using selection techniques like tournament selection or roulette wheel selection.

4. *Crossover*: Perform crossover (recombination) on the selected permutations to create new offspring permutations.

5. *Mutation*: Introduce random changes (mutations) in the offspring permutations to maintain diversity in the population.

6. *Fitness Evaluation for the new individuals*: Evaluate the fitness of the new population.

7. *Form the new population*: Select the surviving individuals based on scores, with chances direct proportional with their performance.

8. Repeat steps 3-7 for a certain number of generations or until a termination condition is met (e.g., a maximum number of iterations or a satisfactory solution is found).


9. *Termination*: Return the best-performing individual (permutation) found as the solution to the problem.

Note: The fitness function used in this problem should calculate the total weight of the queens based on the positions specified by the permutation. Additionally, the fitness function should penalize solutions that violate the non-threatening constraint by assigning a lower fitness score to such permutations.

In [60]:
from random import randint, random, sample
import numpy as np


# Create the individual function
def individual(N):
    """
    Create a member of the population
    :param N: the number of queens on the N x N board
    :return: the sampled permutation
    """
    return sample(range(1, N + 1), N)


In [61]:
# Create the Population function
def populate(N, count):
    return [individual(N) for _ in range(count)]


In [62]:
# Return the sum of the weighted positions and see which one fits better
def fitness_evaluation(weighted_board, population):
    fitness_scores = []
    for specific_set in population:
        if filter_queen_sample(specific_set, len(specific_set)):
            fitness_scores.append(sum(weighted_board[index, queen - 1] for index, queen in enumerate(specific_set)))
        else:
            fitness_scores.append(0)  # Penalize invalid solutions
    return fitness_scores


In [63]:
def filter_queen_sample(rand_sample, N):
    bad_position = []
    for row_index, queen_position in enumerate(rand_sample, 1):
        if (row_index, queen_position) in bad_position:
            return False
        i = 1
        while row_index + i <= N:
            bad_position.extend([(row_index + i, queen_position + i), (row_index + i, queen_position - i)])
            i += 1

    # Checks to see if the sample contains unique items or not
    return len(rand_sample) == len(set(rand_sample))

In [64]:
def tournament_selection(population, fitness_scores, k=3):
    selected = []
    for _ in range(len(population)):
        tournament = sample(list(zip(population, fitness_scores)), k)
        selected.append(max(tournament, key=lambda x: x[1])[0])
    return selected


In [65]:
def crossover(parent1, parent2):
    point = randint(1, len(parent1) - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def mutate(individual):
    point1, point2 = sample(range(len(individual)), 2)
    individual[point1], individual[point2] = individual[point2], individual[point1]
    return individual

In [66]:
def genetic_algorithm(N, weighted_board, population_size=100, generations=1000, crossover_rate=0.8, mutation_rate=0.1):
    population = populate(N, population_size)
    best_individual = None
    max_fitness = -float('inf')

    for generation in range(generations):
        fitness_score = fitness_evaluation(weighted_board, population)

        # Look for the best solution
        current_best_fitness = max(fitness_score)
        if current_best_fitness > max_fitness:
            max_fitness = current_best_fitness
            best_individual = population[fitness_score.index(current_best_fitness)]

        selected_population = tournament_selection(population, fitness_score)
        next_generation = []

        for i in range(0, len(selected_population), 2):
            parent1 = selected_population[i]
            parent2 = selected_population[min(i + 1, len(selected_population) - 1)]
            if random() < crossover_rate:
                child1, child2 = crossover(parent1, parent2)
                next_generation.extend([child1, child2])
            else:
                next_generation.extend([parent1, parent2])

        # Mutation
        next_generation = [mutate(current_individual) if random() < mutation_rate else current_individual for current_individual in next_generation]

        population.clear()
        for next_sample in next_generation:
            if filter_queen_sample(next_sample, N):
                population.append(next_sample)

        # We extend the population with the remaining samples until we meat our goal
        population.extend(populate(N, population_size - len(population)))

    return best_individual, max_fitness


# Example usage
N = 8
weighted_board = np.random.rand(N, N)  # Example weighted board
best_individual, best_fitness = genetic_algorithm(N, weighted_board)
print(f"Best individual: {best_individual}")
print(f"Best fitness: {best_fitness}")

Best individual: [4, 7, 5, 2, 6, 1, 3, 8]
Best fitness: 4.90576381294367
